## 패션 이미지 데이터셋 전처리

In [6]:
import pandas as pd

path = 'data/meta/'
# train = pd.read_json(path + 'train_no_dup.json')
# valid = pd.read_json(path + 'valid_no_dup.json')
test = pd.read_json(path + 'test_no_dup.json')
# blank = pd.read_json(path + 'fill_in_blank_test.json')

In [9]:
test[test['set_id']==199221246]

,name,views,items,image,likes,date,set_url,set_id,desc
594,Top Art Set 6.1.16-Where are my Shoes?,705,"[{'index': 1, 'name': 'pre-owned manolo blahni...",http://ak2.polyvoreimg.com/cgi/img-set/cid/199...,183,8 months,http://www.polyvore.com/top_art_set_16-where_a...,199221246,An art collage from May 2016 by easy-dressing ...


## Data Load Example

In [8]:
import pandas as pd
from dataloader.sampling import DataSampler
#from dataloader.multimodal_data import MultiModalData
from dataloader.multilstm_data import MultiModalData
import os

base_dir = os.getcwd()
data_dir = os.path.join(base_dir, 'data')
meta_dir = os.path.join(data_dir, 'meta')
image_dir = os.path.join(data_dir, 'images')
sampler = DataSampler(data_path = meta_dir, k=100, test_sampling_ratio=1)
concat_df, question_data = sampler.sample_data()

train_dataset = MultiModalData(concat_df, sampler.category_df, image_dir, mode='train')
valid_dataset = MultiModalData(concat_df, sampler.category_df, image_dir, mode='valid')
test_dataset = MultiModalData(concat_df, sampler.category_df, image_dir, question = question_data, mode='test')

In [11]:
concat_df

,name,items,desc,set_id,type
0,Casual Eleanor.,"[{'index': 1, 'name': 'doublju women's crew ne...",A fashion look from August 2016 by haley-bob1 ...,205211881,train
1,Who wears short short !?,"[{'index': 1, 'name': 'abercrombie fitch appli...",A fashion look from July 2016 by blancacalleja...,203655321,train
2,Date Night,"[{'index': 1, 'name': 'dip dye raglan blue whi...",A fashion look from February 2017 by hattie4pa...,216829898,train
3,TOP,"[{'index': 1, 'name': 'forever new lauren embe...",A fashion look from December 2013 featuring sh...,107222824,train
4,interview outfit,"[{'index': 1, 'name': 'wool sweater', 'price':...",A fashion look from November 2016 by smartbuyg...,211086877,train
...,...,...,...,...,...
7382,Sheinside.com,"[{'index': 1, 'name': 'black sleeveless hollow...",#Sheinside,122358035,test
7383,Inspired by street style/ the new queen,"[{'index': 1, 'name': 'j.crew jeweled chandeli...",A fashion look from November 2014 featuring pi...,141687053,test
7384,*edgy chic*,"[{'index': 1, 'name': 'crop pullover', 'price'...",A fashion look from February 2014 featuring cr...,113788101,test
7385,Cute White Laser Loose Top Set: Contest Entry,"[{'index': 1, 'name': 'shein sheinside loose f...",A fashion look from June 2016 by krystinekerch...,199807808,test


In [4]:
concat_df['type'].value_counts()

type
train    4714
test     2630
valid     843
Name: count, dtype: int64

In [10]:
concat_df[concat_df['set_id']==199221246]

,name,items,desc,set_id,type


In [23]:
test_dataset[4].keys()

dict_keys(['texts', 'images', 'set_id', 'question', 'answer_texts', 'answer_images'])

In [3]:
test_dataset[100]['answer_images']

c:\ProgramData\Anaconda3\envs\jin\Lib\site-packages\torch\nn\functional.py:5560: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


0


tensor([[-0.3274, -0.2683,  0.6309,  ...,  0.7676,  0.0580, -0.0901],
        [-0.1646, -0.3567,  0.4143,  ...,  0.4258,  0.2651,  0.5854],
        [-0.2900, -0.3955,  0.2117,  ...,  0.7139,  0.1151,  0.0714]],
       device='cuda:0', grad_fn=<ToCopyBackward0>)

In [26]:
len(valid_dataset)

305

## Dataloader

In [29]:
from torch.nn.utils.rnn import pad_sequence
import torch
def collate_fn(batch):
    """
    배치 내 데이터를 패딩 처리하고 필요한 키로 구성된 dictionary 반환.
    """
    image_embeddings = [item['images'] for item in batch]
    seq_embeddings = [item['texts'] for item in batch]
    lengths = torch.tensor([len(seq) for seq in image_embeddings])

    image_embeddings_padded = pad_sequence(image_embeddings, batch_first=True)
    seq_embeddings_padded = pad_sequence(seq_embeddings, batch_first=True)
    mask = (image_embeddings_padded.sum(dim=2) != 0).float()

    return {
        'image_embeddings': image_embeddings_padded,
        'seq_embeddings': seq_embeddings_padded,
        'lengths': lengths,
        'mask': mask
    }

In [32]:
from torch.utils.data import Dataset, DataLoader
#from AI_MMR.model.multilstm import Multifusion, MultiLSTM
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

train_dataloader = DataLoader(
        train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn
    )
valid_dataloader = DataLoader(
        valid_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn
    )
test_dataloader = DataLoader(
        test_dataset, batch_size=4, shuffle=True
    )

In [23]:
test_dataloader

In [33]:
# Dataloader
 
for batch_idx, batch in enumerate(test_dataloader):
    print(batch.keys())
    break

dict_keys(['texts', 'images', 'set_id', 'question'])


In [ ]:
 
for batch_idx, batch in enumerate(valid_dataloader):
    print(batch.shape)
    break

In [4]:
mlp = Multifusion().to(device)
lstm = MultiLSTM(input_size=512, hidden_size=512, num_layers=1, bidirectional=True).to(device)

In [24]:
from AI_MMR.model.multilstm import Multifusion